In [8]:
import sqlite3
import pandas as pd

In [9]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [10]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [11]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [12]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [13]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [74]:
tasks+=1
pd.read_sql(
    '''   
SELECT 'CRIM' AS Column_Name, COUNT(*) - COUNT(CRIM) AS Null_Count FROM boston
UNION ALL
SELECT 'ZN', COUNT(*) - COUNT(ZN) FROM boston
UNION ALL
SELECT 'INDUS', COUNT(*) - COUNT(INDUS) FROM boston
UNION ALL
SELECT 'CHAS', COUNT(*) - COUNT(CHAS) FROM boston
UNION ALL
SELECT 'NOX', COUNT(*) - COUNT(NOX) FROM boston;
    ''',
    con,
)

,Column_Name,Null_Count
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [75]:
tasks+=1
pd.read_sql(
    '''
SELECT 'CRIM' AS Column_Name, COUNT(DISTINCT CRIM) AS Unique_Count FROM boston
UNION ALL
SELECT 'ZN' AS Column_Name, COUNT(DISTINCT ZN) AS Unique_Count FROM boston
UNION ALL
SELECT 'INDUS' AS Column_Name, COUNT(DISTINCT INDUS) AS Unique_Count FROM boston
UNION ALL
SELECT 'CHAS' AS Column_Name, COUNT(DISTINCT CHAS) AS Unique_Count FROM boston
UNION ALL
SELECT 'NOX' AS Column_Name, COUNT(DISTINCT NOX) AS Unique_Count FROM boston;
    ''',
    con,
)

,Column_Name,Unique_Count
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [76]:
tasks+=1
pd.read_sql(
    '''
         SELECT
        (SELECT (max(CRIM)+min(CRIM))/ 2 FROM boston) AS mid_CRIM,
        (SELECT (max(ZN)+min(ZN))/ 2 FROM boston) AS mid_ZN,
        (SELECT (max(INDUS)+min(INDUS))/ 2 FROM boston) AS mid_INDUS,
        (SELECT (max(CHAS)+min(CHAS))/ 2 FROM boston) AS mid_CHAS,
        (SELECT (max(NOX)+min(NOX))/ 2 FROM boston) AS mid_NOX
    FROM boston
    LIMIT 1;
    ''',
    con,
)

#CRIM - уровень преступности на душу населения по городам очень высокий, а доля жилой земли низкая

,mid_CRIM,mid_ZN,mid_INDUS,mid_CHAS,mid_NOX
0,44.49126,50.0,14.1,0.5,0.628


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [66]:
tasks+=1
pd.read_sql(
    '''
WITH MostExpensive AS (
  SELECT AVG(RM) AS AvgRMExpensive
  FROM (
    SELECT RM
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1
  )
),
CheapestGroups AS (
  SELECT
    COUNT(*) OVER () AS NumHouses,
    AVG(RM) OVER (ORDER BY MEDV ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS AvgRMCheapest
  FROM boston
),
Differences AS (
  SELECT
    NumHouses,
    AvgRMCheapest,
    (SELECT AvgRMExpensive FROM MostExpensive) AS AvgRMExpensive,
    (SELECT AvgRMExpensive FROM MostExpensive) - AvgRMCheapest AS Difference
  FROM CheapestGroups
  WHERE NumHouses IN (25, 50, 100, 200, 300)
)
SELECT * FROM Differences
ORDER BY NumHouses;


    ''',
    con,
)

,NumHouses,AvgRMCheapest,AvgRMExpensive,Difference


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [73]:
tasks+=1
pd.read_sql(
    '''

SELECT LSTAT, MEDV, rank () over (order by MEDV desc) as rank
FROM boston
group by MEDV 




    ''',
    con,
)

# в домах с самой дорогой стоимостью более низкий процент населения

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,5.91,48.8,2
2,3.81,48.5,3
3,3.95,48.3,4
4,3.92,46.7,5
...,...,...,...
224,30.81,7.2,225
225,36.98,7.0,226
226,29.97,6.3,227
227,26.77,5.6,228


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [68]:
tasks+=1
pd.read_sql(
    '''
SELECT avg(MEDV) as avg_price,
case
WHEN CHAS = 0 THEN "не граничит"
WHEN CHAS = 1 THEN 'граничит'
END as CHAS
FROM boston
where CHAS = 1
union all
SELECT avg(MEDV) as avg_price,
case
WHEN CHAS = 0 THEN "не граничит"
WHEN CHAS = 1 THEN 'граничит'
END as CHAS
FROM boston
where CHAS = 0


    ''',
    con,
)

,avg_price,CHAS
0,28.440000,граничит
1,22.093843,не граничит





7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [69]:
tasks+=1
pd.read_sql(
    '''
   SELECT
  'CRIM' AS avg_CRIM,
  AVG(CASE WHEN CHAS = 1 THEN CRIM ELSE NULL END) AS avg_river,
  AVG(CASE WHEN CHAS = 0 THEN CRIM ELSE NULL END) AS avg_no_river
FROM boston
HAVING avg_river > avg_no_river
UNION ALL
SELECT
  'ZN' AS avg_ZN,
  AVG(CASE WHEN CHAS = 1 THEN ZN ELSE NULL END) AS avg_river,
  AVG(CASE WHEN CHAS = 0 THEN ZN ELSE NULL END) AS avg_no_river
FROM boston
group by ZN
HAVING avg_river > avg_no_river
UNION ALL
SELECT
  'INDUS' AS avg_INDUS,
  AVG(CASE WHEN CHAS = 1 THEN INDUS ELSE NULL END) AS avg_river,
  AVG(CASE WHEN CHAS = 0 THEN INDUS ELSE NULL END) AS avg_no_river
FROM boston
HAVING avg_river > avg_no_river
UNION ALL
SELECT
  'NOX' AS avg_NOX,
  AVG(CASE WHEN CHAS = 1 THEN NOX ELSE NULL END) AS avg_river,
  AVG(CASE WHEN CHAS = 0 THEN NOX ELSE NULL END) AS avg_no_river
FROM boston
HAVING avg_river > avg_no_river;
    
    ''',
    con,
)

,avg_CRIM,avg_river,avg_no_river
0,INDUS,12.719143,11.019193
1,NOX,0.593426,0.551817


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [47]:
tasks+=1
pd.read_sql(
    '''
WITH Percentiles AS (
  SELECT
    INDUS,
    NOX,
    NTILE(10) OVER (ORDER BY INDUS) AS IndusPercentile,
    NTILE(10) OVER (ORDER BY NOX) AS NoxPercentile
  FROM boston
)
SELECT
  DISTINCT IndusPercentile * 10 AS percentile,
  MAX(CASE WHEN IndusPercentile = NoxPercentile THEN INDUS END)
    OVER (PARTITION BY IndusPercentile) AS INDUS,
  MAX(CASE WHEN IndusPercentile = NoxPercentile THEN NOX END)
    OVER (PARTITION BY NoxPercentile) AS NOX
FROM Percentiles
ORDER BY percentile;



    ''',
    con,
)

#концентрация оксидов азота варируется 0,4 до 0,8 не зависимо от доли акров

,percentile,INDUS,NOX
0,10,2.68,0.422
1,10,2.68,0.442
2,10,2.68,0.453
3,10,2.68,0.507
4,10,2.68,0.538
5,20,4.39,0.422
6,20,4.39,0.442
7,20,4.39,0.453
8,20,4.39,0.507
9,20,4.39,0.538


In [48]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены не все задания! 🙀
